<a href="https://colab.research.google.com/github/26082005-m/innomatics/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages (run in Colab)
!pip install streamlit opencv-python-headless pandas openpyxl numpy

# Save this code in a file called app.py
import streamlit as st
import cv2
import numpy as np
import pandas as pd
from io import BytesIO

st.set_page_config(page_title="OMR Evaluation System", layout="wide")
st.title("📄 Automated OMR Evaluation System")

# Upload OMR images
uploaded_files = st.file_uploader(
    "Upload OMR Images (JPEG/PNG)", type=['jpg', 'jpeg', 'png'], accept_multiple_files=True
)

# Upload answer key Excel
answer_key_file = st.file_uploader(
    "Upload Answer Key Excel", type=['xlsx']
)

def preprocess_image(image_bytes):
    # Convert bytes to OpenCV image
    file_bytes = np.asarray(bytearray(image_bytes.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_GRAYSCALE)
    # Thresholding for bubble detection
    _, thresh = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV)
    return thresh

def detect_bubbles(thresh_img, num_questions=10, num_options=4):
    # Simple contour detection
    contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bubble_contours = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if 10 < w < 50 and 10 < h < 50:  # filter approx bubble size
            bubble_contours.append((x, y, w, h))
    bubble_contours = sorted(bubble_contours, key=lambda b: (b[1], b[0]))  # top-to-bottom, left-to-right
    answers = []
    for q in range(num_questions):
        options = bubble_contours[q*num_options:(q+1)*num_options]
        filled = None
        for i, (x, y, w, h) in enumerate(options):
            roi = thresh_img[y:y+h, x:x+w]
            if cv2.countNonZero(roi) > (w*h*0.5):  # more than 50% filled
                filled = i
        answers.append(filled)
    return answers

def evaluate_omr(omr_answers, answer_key):
    scores = {}
    for subject in answer_key.columns:
        correct = 0
        for i, key in enumerate(answer_key[subject]):
            if omr_answers[i] == ord(key.lower()) - 97:  # convert 'a'->0
                correct += 1
        scores[subject] = correct
    return scores

if uploaded_files and answer_key_file:
    # Read answer key Excel
    answer_key_df = pd.read_excel(answer_key_file)
    all_results = []

    st.subheader("OMR Evaluation Results")

    for uploaded_file in uploaded_files:
        thresh = preprocess_image(uploaded_file)
        # Assume each OMR sheet has same number of questions as answer key rows
        omr_answers = detect_bubbles(thresh, num_questions=len(answer_key_df), num_options=4)
        scores = evaluate_omr(omr_answers, answer_key_df)
        scores['Image'] = uploaded_file.name
        all_results.append(scores)

    result_df = pd.DataFrame(all_results)
    result_df = result_df[['Image'] + [col for col in result_df.columns if col != 'Image']]
    st.dataframe(result_df)

    # Download CSV
    csv = result_df.to_csv(index=False).encode('utf-8')
    st.download_button(
        label="Download Results as CSV",
        data=csv,
        file_name='OMR_results.csv',
        mime='text/csv'
    )


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.0 MB/s eta 0:00:00


2025-09-21 06:34:57.390 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-21 06:34:57.391 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-21 06:34:57.487 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-21 06:34:57.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-21 06:34:57.490 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-21 06:34:57.490 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-21 06:34:57.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [3]:
# 1️⃣ Save the Streamlit code into a file called app.py
code = """
import streamlit as st
import cv2
import numpy as np
import pandas as pd
from io import BytesIO

st.set_page_config(page_title="OMR Evaluation System", layout="wide")
st.title("📄 Automated OMR Evaluation System")

uploaded_files = st.file_uploader(
    "Upload OMR Images (JPEG/PNG)", type=['jpg', 'jpeg', 'png'], accept_multiple_files=True
)

answer_key_file = st.file_uploader(
    "Upload Answer Key Excel", type=['xlsx']
)

def preprocess_image(image_bytes):
    file_bytes = np.asarray(bytearray(image_bytes.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_GRAYSCALE)
    _, thresh = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV)
    return thresh

def detect_bubbles(thresh_img, num_questions=10, num_options=4):
    contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bubble_contours = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if 10 < w < 50 and 10 < h < 50:
            bubble_contours.append((x, y, w, h))
    bubble_contours = sorted(bubble_contours, key=lambda b: (b[1], b[0]))
    answers = []
    for q in range(num_questions):
        options = bubble_contours[q*num_options:(q+1)*num_options]
        filled = None
        for i, (x, y, w, h) in enumerate(options):
            roi = thresh_img[y:y+h, x:x+w]
            if cv2.countNonZero(roi) > (w*h*0.5):
                filled = i
        answers.append(filled)
    return answers

def evaluate_omr(omr_answers, answer_key):
    scores = {}
    for subject in answer_key.columns:
        correct = 0
        for i, key in enumerate(answer_key[subject]):
            if omr_answers[i] == ord(key.lower()) - 97:
                correct += 1
        scores[subject] = correct
    return scores

if uploaded_files and answer_key_file:
    answer_key_df = pd.read_excel(answer_key_file)
    all_results = []

    st.subheader("OMR Evaluation Results")

    for uploaded_file in uploaded_files:
        thresh = preprocess_image(uploaded_file)
        omr_answers = detect_bubbles(thresh, num_questions=len(answer_key_df), num_options=4)
        scores = evaluate_omr(omr_answers, answer_key_df)
        scores['Image'] = uploaded_file.name
        all_results.append(scores)

    result_df = pd.DataFrame(all_results)
    result_df = result_df[['Image'] + [col for col in result_df.columns if col != 'Image']]
    st.dataframe(result_df)

    csv = result_df.to_csv(index=False).encode('utf-8')
    st.download_button(
        label="Download Results as CSV",
        data=csv,
        file_name='OMR_results.csv',
        mime='text/csv'
    )
"""

with open("app.py", "w") as f:
    f.write(code)


In [5]:
!streamlit run app.py & npx localtunnel --port 8501


⠙

⠹⠸⠼⠴⠦⠧
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.142.221:8501

⠇⠏⠋Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y)   Stopping...
^C


In [6]:
!pip install pyngrok


In [7]:
code = """
# (paste the same Streamlit OMR code here)
"""
with open("app.py", "w") as f:
    f.write(code)


In [9]:
!ngrok authtoken YOUR_NGROK_AUTHTOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [16]:
# 1️⃣ Save your Streamlit app
code = """
# (paste your app.py Streamlit OMR code here)
"""
with open("app.py", "w") as f:
    f.write(code)

# 2️⃣ Install localtunnel
!npm install -g localtunnel

# 3️⃣ Run Streamlit + localtunnel
get_ipython().system_raw("streamlit run app.py --server.port 8501 &")
!lt --port 8501


⠙⠹⠸⠼
changed 22 packages in 566ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼your url is: https://common-webs-work.loca.lt
^C


In [21]:
# Install dependencies
!pip install streamlit opencv-python-headless pandas openpyxl numpy pyngrok -q

# 1️⃣ Write the Streamlit app
app_code = """
import streamlit as st
import cv2
import numpy as np
import pandas as pd

st.title("📄 Automated OMR Evaluation System")

uploaded_files = st.file_uploader(
    "Upload OMR Images (JPEG/PNG)", type=['jpg', 'jpeg', 'png'], accept_multiple_files=True
)

answer_key_file = st.file_uploader(
    "Upload Answer Key Excel", type=['xlsx']
)

def preprocess_image(image_bytes):
    file_bytes = np.asarray(bytearray(image_bytes.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_GRAYSCALE)
    _, thresh = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV)
    return thresh

def detect_bubbles(thresh_img, num_questions=10, num_options=4):
    contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bubble_contours = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if 10 < w < 50 and 10 < h < 50:
            bubble_contours.append((x, y, w, h))
    bubble_contours = sorted(bubble_contours, key=lambda b: (b[1], b[0]))
    answers = []
    for q in range(num_questions):
        options = bubble_contours[q*num_options:(q+1)*num_options]
        filled = None
        for i, (x, y, w, h) in enumerate(options):
            roi = thresh_img[y:y+h, x:x+w]
            if cv2.countNonZero(roi) > (w*h*0.5):
                filled = i
        answers.append(filled)
    return answers

def evaluate_omr(omr_answers, answer_key):
    scores = {}
    for subject in answer_key.columns:
        correct = 0
        for i, key in enumerate(answer_key[subject]):
            # Extract the last character that is a letter (a, b, c, d)
            option = str(key).strip()[-1].lower()  # takes '1 - a' -> 'a'
            if option in ['a', 'b', 'c', 'd']:
                if omr_answers[i] == ord(option) - 97:  # convert 'a'->0
                    correct += 1
        scores[subject] = correct
    return scores


if uploaded_files and answer_key_file:
    answer_key_df = pd.read_excel(answer_key_file)
    all_results = []

    st.subheader("OMR Evaluation Results")

    for uploaded_file in uploaded_files:
        thresh = preprocess_image(uploaded_file)
        omr_answers = detect_bubbles(thresh, num_questions=len(answer_key_df), num_options=4)
        scores = evaluate_omr(omr_answers, answer_key_df)
        scores['Image'] = uploaded_file.name
        all_results.append(scores)

    result_df = pd.DataFrame(all_results)
    result_df = result_df[['Image'] + [col for col in result_df.columns if col != 'Image']]
    st.dataframe(result_df)

    csv = result_df.to_csv(index=False).encode('utf-8')
    st.download_button(
        label="Download Results as CSV",
        data=csv,
        file_name='OMR_results.csv',
        mime='text/csv'
    )
"""

with open("app.py", "w") as f:
    f.write(app_code)

# 2️⃣ Setup ngrok
from pyngrok import ngrok
NGROK_AUTH_TOKEN = "32ncm5WRCmkvJYSOtFQu2zwtSJ1_5oibFkCREgabDz6GbFA7N"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 3️⃣ Run Streamlit in background
get_ipython().system_raw("streamlit run app.py --server.port 8501 &")

# 4️⃣ Open ngrok tunnel
import time
time.sleep(5)  # wait for Streamlit server to start
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://a3f4fccf9a23.ngrok-free.app" -> "http://localhost:8501">